In [15]:
# import essential libraries
import pandas as pd
import numpy as np

# visual
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# import the datetime module
from datetime import datetime

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# set format
pd.options.display.float_format = '{:.2f}'.format

# YouTube

In [2]:
# read in youtube data
youtube = pd.read_csv('youtube_data.csv')

In [3]:
# check shape
youtube.shape

(15730, 9)

In [4]:
youtube.head()

,title,publishedTime,Unnamed: 2,date_ref,pub_date,duration,viewCount,hashtag,category
0,The Truth Behind The Global Pandemic. Jordan M...,18,0,9/16/22,09/16/22,26:40:00,48882,comedy,humor
1,The Best Comedy Special Ever. Gabriel Rutledge...,1,30.5,9/16/22,08/16/22,25:24:00,621694,comedy,humor
2,Look At These Assholes: Trump Family Edition |...,2,30.5,9/16/22,07/17/22,05:45,2248400,comedy,humor
3,More Than Funny Comedy Special | Michael Jr. #...,11,30.5,9/16/22,10/15/21,01:23:38,1813052,comedy,humor
4,2020 FUNNY COMEDY BEST OF MARVELOUS SARAH PRIN...,1,365,9/16/22,09/16/21,28:51:00,21036895,comedy,humor


#### Date of reference: 09/16/2022

In [5]:
# check datatypes
youtube.dtypes

title            object
publishedTime     int64
Unnamed: 2       object
date_ref         object
pub_date         object
duration         object
viewCount        object
hashtag          object
category         object
dtype: object

#### Drop Columns

In [6]:
# drop meaningless columns
youtube.drop(columns = ['publishedTime', 'Unnamed: 2', 'date_ref'], inplace=True)

#### NaNs

In [7]:
# check nulls
youtube.isnull().sum()

title        0
pub_date     0
duration     0
viewCount    0
hashtag      0
category     0
dtype: int64

#### Drop ViewCount letter

In [8]:
# check viewCount datatype
indexname = youtube[youtube.viewCount.str.contains('[A-Za-z]')].index

In [9]:
# drop index with letter as viewCount
youtube.drop(indexname, inplace=True)

In [10]:
# it's not showing stats so there's other non-integer type in this column
youtube.viewCount.describe()

count     15678
unique    11517
top           2
freq         25
Name: viewCount, dtype: object

In [11]:
# check shape after dropping views with letter
youtube.shape

(15678, 6)

#### Replace ',' in viewCount

In [12]:
# replace ','
youtube.viewCount = youtube.viewCount.str.replace(',','')

In [13]:
# change viewcount to int
youtube.viewCount = youtube.viewCount.astype(int)

In [14]:
# check viewcount stats
youtube.viewCount.describe()

count        15678.00
mean       2294596.36
std       18669485.26
min              1.00
25%           1347.00
50%           6870.00
75%         219229.00
max     1626628893.00
Name: viewCount, dtype: float64

#### Convert pub_date to datetime

In [16]:
youtube['pub_date']=pd.to_datetime(youtube['pub_date'], errors='coerce')

In [17]:
# check nulls
youtube.isnull().sum()

title        0
pub_date     1
duration     0
viewCount    0
hashtag      0
category     0
dtype: int64

In [18]:
# drop nulls
youtube.dropna(inplace = True)

In [19]:
# check shape
youtube.shape

(15677, 6)

In [20]:
youtube.pub_date

0       2022-09-16
1       2022-08-16
2       2022-07-17
3       2021-10-15
4       2021-09-16
           ...    
15725   2022-09-13
15726   2022-09-15
15727   2022-09-15
15728   2022-09-13
15729   2022-09-16
Name: pub_date, Length: 15677, dtype: datetime64[ns]

#### Convert 'duration' to seconds

In [21]:
def change_time(string):
    if len(string)>5:
        if string.endswith(':00'):
            string = string.replace(':00', '')
    return string

In [22]:
# apply function to convert format
youtube.duration = youtube.duration.apply(change_time)

In [23]:
# check result
youtube[(youtube.duration.str.endswith(':00')) & (youtube.duration.str.len()>5)]

,title,pub_date,duration,viewCount,hashtag,category


In [24]:
youtube.duration

0           26:40
1           25:24
2           05:45
3        01:23:38
4           28:51
           ...   
15725       00:16
15726       00:43
15727       00:14
15728       00:39
15729       04:46
Name: duration, Length: 15677, dtype: object

In [25]:
# this need to apply twice
def add_zero(string):
    if len(string)<=5:
        string = '00:' + string
    return string

In [26]:
youtube.duration = youtube.duration.apply(add_zero)
youtube.duration = youtube.duration.apply(add_zero)

In [27]:
# check result, now length should be universal (8)
youtube.duration

0        00:26:40
1        00:25:24
2        00:05:45
3        01:23:38
4        00:28:51
           ...   
15725    00:00:16
15726    00:00:43
15727    00:00:14
15728    00:00:39
15729    00:04:46
Name: duration, Length: 15677, dtype: object

In [28]:
# double check length
youtube[youtube.duration.str.len()<8]

,title,pub_date,duration,viewCount,hashtag,category


In [29]:
# convert hour, minute, second
youtube['hour'] = youtube['duration'].str.slice(0,2)
youtube.hour = youtube.hour.astype(int)
youtube['minute'] =  youtube['duration'].str.slice(3,5)
youtube.minute = youtube.minute.astype(int)
youtube['second'] =  youtube['duration'].str.slice(6,8)
youtube.second = youtube.second.astype(int)
youtube['length'] = 3600*youtube.hour + 60*youtube.minute + youtube.second

In [30]:
# combined function
def change_time(string):
    if len(string)>5:
        if string.endswith(':00'):
            string = string.replace(':00', '')
    return string

def add_zero(string):
    if len(string)<=5:
        string = '00:' + string
    return string

def prep_youtube(youtube):
    # drop useless columns
    youtube.drop(columns = ['publishedTime', 'Unnamed: 2', 'date_ref'], inplace=True)
    # drop non-number viewCount rows
    indexname = youtube[youtube.viewCount.str.contains('[A-Za-z]')].index
    # drop index with letter as viewCount
    youtube.drop(indexname, inplace=True)
    # replace ',' in viewCount
    youtube.viewCount = youtube.viewCount.str.replace(',','')
    # change viewcount to int
    youtube.viewCount = youtube.viewCount.astype(int)
    # convert datetime
    youtube['pub_date']=pd.to_datetime(youtube['pub_date'], errors='coerce')
    # drop nulls after date conversion
    youtube.dropna(inplace = True)
    # convert duration to length
    youtube.duration = youtube.duration.apply(change_time)
    # add zero to have a universal length
    youtube.duration = youtube.duration.apply(add_zero)
    youtube.duration = youtube.duration.apply(add_zero)
    # split time into hour, minute, second
    youtube['hour'] = youtube['duration'].str.slice(0,2)
    youtube['minute'] =  youtube['duration'].str.slice(3,5)
    youtube['second'] =  youtube['duration'].str.slice(6,8)
    # convert datatype
    youtube.hour = youtube.hour.astype(int)
    youtube.minute = youtube.minute.astype(int)
    youtube.second = youtube.second.astype(int)
    # calculate length
    youtube['length(second)'] = 3600*youtube.hour + 60*youtube.minute + youtube.second
    
    return youtube

***

# TikTok

In [55]:
# read in combined data from 5 categories
tiktok = pd.read_csv('tiktok_data.csv')

In [56]:
# check shaoe
tiktok.shape

(21598, 12)

In [57]:
tiktok.head()

,commentCount,diggCount,playCount,shareCount,followerCount,heartCount,videoCount,description,duration,time,hashtag,category
0,33000,5100000,46600000,16400,2800000,58800000,318,Can you count how many layers are in this dres...,40,1626635893,fashion,fashion
1,0,5100000,33200000,21900,163500,10700000,451,it had to be done ‚ùå #SmellLikeIrishSpring #O...,59,1648504875,fashion,fashion
2,54600,5000000,21300000,17100,3800000,81900000,510,Reply to @wibowowowo_17 NGL this was tougher t...,17,1594775651,fashion,fashion
3,18300,5000000,35100000,27300,6500000,120900000,458,al principio pens√© que se equivocaron de paqu...,75,1655771466,fashion,fashion
4,15100,4900000,12900000,9574,241800,8500000,72,I love this outfit so much #fashion #fashionti...,42,1642621639,fashion,fashion


#### Rename Columns

In [58]:
# rename columns
tiktok.rename(columns = {'commentCount':'comments', 'diggCount':'likes',
                     'playCount':'views', 'shareCount':'shares', 'time':'epoch',
                    'followerCount': 'total_followers',
                    'heartCount':'total_likes',
                    'videoCount': 'total_videos'}, inplace=True)

#### Convert Time

In [59]:
# convert epoch time
tiktok['date'] = (pd.to_datetime(tiktok['epoch'], unit='s')
                     .dt.tz_localize('utc')
                     .dt.tz_convert('US/Central'))

In [60]:
tiktok.date = tiktok.date.astype(str).str.slice(0,10)

#### Convert Date

In [61]:
tiktok.date = pd.to_datetime(tiktok.date)

#### Drop Column

In [62]:
# drop column
tiktok.drop(columns = 'epoch', inplace = True)

#### Drop Duration ==0 Index

In [63]:
# locate duration = 0 columns
indexname = tiktok[tiktok.duration ==0].index
# drop rows with duration = 0
tiktok.drop(indexname, inplace=True)

#### Create Condition Column

In [43]:
# create conditions
conditions = [(tiktok['duration']<=15),
              (tiktok['duration']>15)&(tiktok['duration']<=60),
              (tiktok['duration']>60)&(tiktok['duration']<=180), 
              (tiktok['duration']>180)]
# condition values
values = ['Short: 0-15s', 'Medium: 15-60s', 'Long: 1-3mins', 'Extra-long: >3mins']
# video duration
tiktok['length'] = np.select(conditions, values)

In [44]:
tiktok['category'].replace({'fashion': 'Fashion', 'fitness & lifestyle': 'Fitness & Lifestyle', 'food': 'Food', 'humor': 'Humor', 'political': 'Political'}, inplace=True)

In [46]:
def prep_tiktok(df):
    '''
    This funciton takes in messy tiktok data and return the cleaned version
    '''
    #df.drop(columns = 'Unnamed: 0', inplace = True)
    df.rename(columns = {'commentCount':'comments', 'diggCount':'likes',
                     'playCount':'views', 'shareCount':'shares', 'time':'epoch',
                    'followerCount': 'total_followers',
                    'heartCount':'total_likes',
                    'videoCount': 'total_videos'}, inplace=True)
    df['date'] = (pd.to_datetime(df['epoch'], unit='s')
                     .dt.tz_localize('utc')
                     .dt.tz_convert('US/Central'))
    df['date'] = df['date'].astype(str).str.slice(0,10)
    # convert date
    df.date = pd.to_datetime(df.date)
    
    df.drop(columns = 'epoch', inplace = True)
    # locate duration = 0 columns
    indexname = df[df.duration ==0].index
    # drop rows with duration = 0
    df.drop(indexname, inplace=True)
    
    # df.set_index('date', inplace=True)
    # df.sort_index()
    df['category'].replace({
        'fashion': 'Fashion',
        'fitness & lifestyle': 'Fitness & Lifestyle',
        'food': 'Food',
        'humor': 'Humor',
        'political': 'Political'}, inplace=True)
    # create conditions
    conditions = [(df['duration']<=15),
              (df['duration']>15)&(df['duration']<=60),
              (df['duration']>60)&(df['duration']<=180), 
              (df['duration']>180)]
    values = ['Short: 0-15s', 'Medium: 15-60s', 'Long: 1-3mins', 'Extra-long: >3mins']
    # create length column using conditions
    df['length'] = np.select(conditions, values)

    return df